# CAMeL tools pipeline

```bash
pip install camel_tools
camel_data -i disambig-mle-calima-msa-r13
```

### ازالة التشكيل

In [1]:

# Example Arabic text
# text = "وَالْعَصْرِ إِنَّ الْإِنْسَانَ لَفِي خُسْرٍ"

text = ".يجب ألا تنطبق على مالك البطاقة شروط الاستبعاد من التموين. يجب أن يكون المواطن عضوًا فى البطاقة وليس رب الأسرة، لأن رب الأسرة لا يمكن فصله"


from camel_tools.utils.dediac import dediac_ar

clean_text = dediac_ar(text)


print(clean_text)


.يجب ألا تنطبق على مالك البطاقة شروط الاستبعاد من التموين. يجب أن يكون المواطن عضوا فى البطاقة وليس رب الأسرة، لأن رب الأسرة لا يمكن فصله


### Normalization
compining every thing in one function

In [2]:
from camel_tools.utils.normalize import normalize_unicode, normalize_alef_maksura_ar, normalize_alef_ar, normalize_teh_marbuta_ar

def normalize_arabic_text(text):
    """
    Normalizes Arabic text by:
    1. Converting أ/إ/آ to ا [[3]]
    2. Normalizing ي to ى where appropriate [[3]]
    3. Handling common Unicode character variations [[9]]
    4. Removing diacritics (optional, requires separate function)
    """
    # Step 1: Basic Unicode normalization
    normalized_text = normalize_unicode(text)
    
    # Step 2: Apply specific character normalizations
    normalized_text = normalize_alef_ar(normalized_text)    # Convert إأآ to ا
    normalized_text = normalize_alef_maksura_ar(normalized_text)  # Normalize ي/ى
    normalized_text = normalize_teh_marbuta_ar(normalized_text)   # Normalize ة to ه
    
    return normalized_text


In [3]:
norm_text = normalize_arabic_text(clean_text)
print(norm_text)

.يجب الا تنطبق علي مالك البطاقه شروط الاستبعاد من التموين. يجب ان يكون المواطن عضوا في البطاقه وليس رب الاسره، لان رب الاسره لا يمكن فصله


### tokenization
this is a helper function instead of using `split` for the next step which is the actual tokanization step

In [4]:
from camel_tools.tokenizers.word import simple_word_tokenize

tokens = simple_word_tokenize(text)
print(tokens)

['.', 'يجب', 'ألا', 'تنطبق', 'على', 'مالك', 'البطاقة', 'شروط', 'الاستبعاد', 'من', 'التموين', '.', 'يجب', 'أن', 'يكون', 'المواطن', 'عضوًا', 'فى', 'البطاقة', 'وليس', 'رب', 'الأسرة', '،', 'لأن', 'رب', 'الأسرة', 'لا', 'يمكن', 'فصله']


Need for Morphological Analysis: Effective Arabic tokenization often requires a degree of morphological analysis to separate these clitics (attached particles) from the core word. For example, the word "وبكتابهم" (wa bi-kitābihim - "and with their book") might need to be tokenized into ["و" , "بـ" , "كتاب" , "ـهم"].

##### using D3 tokenizer
- we first get `MLEDisambiguator` which analyse the word and pre-train it on `disambig-mle-calima-msa-r13` MLE model for disambiguating Modern Standard Arabic, (MLE disambiguator that uses a Maximum Likelihood Estimation model.)
- then we make a tokenizer instance then tokenize the tokens from the `simple_word_tokenize`
> note that any tokenizer give better results when geven the word diacritized "مشكله" and result in words that is undiacritized


**it's too slow i will try another type**


In [5]:
from camel_tools.tokenizers.morphological import MorphologicalTokenizer
from camel_tools.disambig.mle import MLEDisambiguator

mle_msa = MLEDisambiguator.pretrained('calima-msa-r13')

# Initialize the D3 tokenizer for MSA
msa_d3_tokenizer = MorphologicalTokenizer(disambiguator=mle_msa, scheme='d3tok',split=True)

# Tokenize the text (includes preprocessing steps like normalization)
d3_tokens = msa_d3_tokenizer.tokenize(tokens)
print(d3_tokens)

['.', 'يجب', 'أن', '+لا', 'تنطبق', 'على', 'مالك', 'ال+', 'بطاقة', 'شروط', 'ال+', 'استبعاد', 'من', 'ال+', 'تموين', '.', 'يجب', 'أن', 'يكون', 'ال+', 'مواطن', 'عضوا', 'في', 'ال+', 'بطاقة', 'و+', 'ليس', 'رب', 'ال+', 'أسرة', '،', 'ل+', 'أن', 'رب', 'ال+', 'أسرة', 'لا', 'يمكن', 'فصل', '+ه']


##### ATB tokenization
- it's pretty fast and it do what i want
- i removed the '+' from the tokens

In [6]:
msa_atb_tokenizer = MorphologicalTokenizer(disambiguator=mle_msa, scheme='atbtok',split=True)
msa_atb_tok = msa_atb_tokenizer.tokenize(tokens)

cleaned_msa_atb_tok = [token.replace('+', '') for token in msa_atb_tok]

print('ATB tokenization (MSA):', msa_atb_tok)


ATB tokenization (MSA): ['.', 'يجب', 'أن', '+لا', 'تنطبق', 'على', 'مالك', 'البطاقة', 'شروط', 'الاستبعاد', 'من', 'التموين', '.', 'يجب', 'أن', 'يكون', 'المواطن', 'عضوا', 'في', 'البطاقة', 'و+', 'ليس', 'رب', 'الأسرة', '،', 'ل+', 'أن', 'رب', 'الأسرة', 'لا', 'يمكن', 'فصل', '+ه']


i filtered out any token that it's length is not grater than 2

In [7]:
filtered_norm_text = [normalize_arabic_text(i) for i in cleaned_msa_atb_tok if len(i) > 2]
print(filtered_norm_text)

['يجب', 'تنطبق', 'علي', 'مالك', 'البطاقه', 'شروط', 'الاستبعاد', 'التموين', 'يجب', 'يكون', 'المواطن', 'عضوا', 'البطاقه', 'ليس', 'الاسره', 'الاسره', 'يمكن', 'فصل']


instead of this i will use stopword removal

## Testing with `NLTK`

In [8]:
from nltk.stem import ISRIStemmer

tokens = simple_word_tokenize(text)
print("before:           ", text)

st = ISRIStemmer()
print('ISRIStemmer norm: ', ' '.join([st.norm(i) for i in tokens]))

print('camel tools norm: ',norm_text)
print('ATB tokenization: ',' '.join(cleaned_msa_atb_tok))

before:            .يجب ألا تنطبق على مالك البطاقة شروط الاستبعاد من التموين. يجب أن يكون المواطن عضوًا فى البطاقة وليس رب الأسرة، لأن رب الأسرة لا يمكن فصله
ISRIStemmer norm:  . يجب الا تنطبق على مالك البطاقة شروط الاستبعاد من التموين . يجب ان يكون المواطن عضوا فى البطاقة وليس رب الأسرة ، لأن رب الأسرة لا يمكن فصله
camel tools norm:  .يجب الا تنطبق علي مالك البطاقه شروط الاستبعاد من التموين. يجب ان يكون المواطن عضوا في البطاقه وليس رب الاسره، لان رب الاسره لا يمكن فصله
ATB tokenization:  . يجب أن لا تنطبق على مالك البطاقة شروط الاستبعاد من التموين . يجب أن يكون المواطن عضوا في البطاقة و ليس رب الأسرة ، ل أن رب الأسرة لا يمكن فصل ه
